In [2]:
class Constraint_violated(Exception):
    pass

def group_sub_ranges(sub_ranges_km, lower_bound_km, upper_bound_km):
    if len(sub_ranges_km) == 0:
        raise ValueError('sub_ranges_km is empty')
    
    found = True
    while found:
        # We will alysws work on sorted list of subranges
        sub_ranges_km.sort()
        
        # Select the shortest subrange
        shortest_sub_range_km = sub_ranges_km.pop(0)
        for k in range(len(sub_ranges_km) - 1, -1, -1):
            # Try to select the largest subrange where the product is still elow the upper bound
            if shortest_sub_range_km * sub_ranges_km[k] < upper_bound_km:
                # If we have found one, remove the term from the list, and push the product into the list
                new_sub_range_km = shortest_sub_range_km * sub_ranges_km.pop(k)
                sub_ranges_km.append(new_sub_range_km)
                break
        else:
            # If we have found no suitable term, push back the shortes subrange into the begining of the list
            sub_ranges_km.insert(0, shortest_sub_range_km)
            found = False
    
    # Check for constarin violation
    for sub_range_km in sub_ranges_km:
        if sub_range_km <= lower_bound_km or sub_range_km >= upper_bound_km:
            raise Constraint_violated
    
    return sub_ranges_km

speed_of_light_km_p_s = 299792.458

def calc_range_from_prf(prf_Hz):
    return speed_of_light_km_p_s / (2 * prf_Hz)

def calc_prf_from_range(range_km):
    return speed_of_light_km_p_s / (2 * range_km)

def design(pulse_width_s, upper_bound_Hz, max_range_km):
    lower_bound_Hz = 1 / (4 * pulse_width_s)
    
    # Convert constrains on frequencies into ranges
    # a < f < b <=> range(b) < range(f) < range(a)
    lower_bound_km = calc_range_from_prf(upper_bound_Hz)
    upper_bound_km = calc_range_from_prf(lower_bound_Hz)
    
    while True:
        try:
            # Factorize the maximum range (FTA)
            sub_ranges_km = list(map(lambda x: x[0] ** x[1], list(factor(max_range_km))))
            
            # Try to group the subranges together
            sub_ranges_km = group_sub_ranges(sub_ranges_km, lower_bound_km, upper_bound_km)
            
            return list(map(calc_prf_from_range, sub_ranges_km))
        except Constraint_violated:
            # If we have found no grouping, we increase the range with 1 and start over
            max_range_km += 1

def calc_apparent_range_from_apparent_delay(apparent_delays_s):
    return (speed_of_light_km_p_s * apparent_delays_s) / 2

def calc_range_from_apparent_delays(apparent_delays_s, prfs_Hz):
    apparent_ranges = list(map(lambda x: Integer(calc_apparent_range_from_apparent_delay(x)), apparent_delays_s))
    sub_ranges_km = list(map(lambda x: Integer(calc_range_from_prf(x)), prfs_Hz))
    maximum_range_km = prod(sub_ranges_km)
    return crt(apparent_ranges, sub_ranges_km) % maximum_range_km

import unittest
test_runner = unittest.TextTestRunner(stream=sys.stdout, verbosity=2)

class Test_group_sub_ranges(unittest.TestCase):
    def test_empty_input(self):
        with self.assertRaises(ValueError):
            group_sub_ranges([], 1, 2)
    
    def test_one_valid_range(self):
        self.assertEqual(group_sub_ranges([2], 1, 3), [2])
    
    def test_two_valid_range_with_valid_product(self):
        self.assertEqual(group_sub_ranges([2, 3], 1, 7), [6])

    def test_three_valid_range_with_and_without_valid_product(self):
        self.assertEqual(group_sub_ranges([2, 3, 4], 1, 7), [4, 6])

    def test_invalid_range(self):
        with self.assertRaises(Constraint_violated):
            group_sub_ranges([2], 2, 7)

        with self.assertRaises(Constraint_violated):
            group_sub_ranges([7], 2, 7)

        with self.assertRaises(Constraint_violated):
            group_sub_ranges([3,7, 4], 2, 7)

    def test_three_valid_range_without_valid_product(self):
        self.assertEqual(group_sub_ranges([2, 3, 4], 1, 5), [2, 3, 4])

test_runner.run(unittest.defaultTestLoader.loadTestsFromTestCase(Test_group_sub_ranges))

class Test_design(unittest.TestCase):
    def test_no_range_increase_needed(self):
        pulse_width_s = 6 / (2*speed_of_light_km_p_s) # range: 6
        upper_bound_Hz = speed_of_light_km_p_s / 2 # range: 1
        
        self.assertEqual(design(pulse_width_s, upper_bound_Hz, 4), list(map(calc_prf_from_range, [4])))

    def test_range_increase_needed(self):
        pulse_width_s = 6 / (2*speed_of_light_km_p_s) # range: 6
        upper_bound_Hz = speed_of_light_km_p_s / 2 # range: 1
        
        self.assertEqual(design(pulse_width_s, upper_bound_Hz, 7), list(map(calc_prf_from_range, [2, 5])))

test_runner.run(unittest.defaultTestLoader.loadTestsFromTestCase(Test_design))

test_empty_input (__main__.Test_group_sub_ranges) ... 

ok
test_invalid_range (__main__.Test_group_sub_ranges) ... 

ok
test_one_valid_range (__main__.Test_group_sub_ranges) ... 

ok
test_three_valid_range_with_and_without_valid_product (__main__.Test_group_sub_ranges) ... 

ok
test_three_valid_range_without_valid_product (__main__.Test_group_sub_ranges) ... 

ok
test_two_valid_range_with_valid_product (__main__.Test_group_sub_ranges) ... 

ok

----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK
test_no_range_increase_needed (__main__.Test_design) ... 

ok
test_range_increase_needed (__main__.Test_design) ... 

ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [6]:
print(list(map(lambda x: x[0] ** x[1], list(factor(456)))))

[8, 3, 19]
